# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [ ]:
'''
Your Code
'''

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import time
import pandas as pd
import _thread

In [2]:
def get_date_post(text):
    regex = "\d+/\d+/\d+"
    pattern = re.compile(regex)
    result = re.search(pattern, text)
    date_post = result.group(0)
    return date_post

In [3]:
def get_news(url, date_select, idx, startTime):    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    columns = ["Date", "Tag", "Title"]
    df_infos = pd.DataFrame(columns=columns)
    columns = ["Date", "Tag", "Title"]
    df_infos = pd.DataFrame(columns=columns)
    for d in soup.find(class_="part_list_2").find_all('h3')[1:]:
        date_post = get_date_post(d.find(class_="date").text)
        if date_post == date_select:
            tag = d.find("em").text
            title = d.find("a").text
            df_info = pd.DataFrame([[date_post, tag, title]], columns=columns)
            df_infos = df_infos.append(df_info, ignore_index=True)
    
    #print(df_infos[:1])
    print("[No.{}]  accumulated time: {}".format(idx, time.time() - startTime))

In [4]:
urls = ['https://www.ettoday.net/news/news-list.htm'] * 5
dates = [datetime.date.today()] * 5
dates = [str(date).replace('-','/') for date in dates]

In [5]:
print("w/o multi-thread:\n")

startTime = time.time()

for idx, (url, date) in enumerate(zip(urls, dates)):
    get_news(url, date, idx, startTime)

w/o multi-thread:

[No.0]  accumulated time: 0.5192501544952393
[No.1]  accumulated time: 1.0460641384124756
[No.2]  accumulated time: 1.5956084728240967
[No.3]  accumulated time: 2.1617605686187744
[No.4]  accumulated time: 2.6934142112731934


In [6]:
print("w/ multi-thread:\n")

startTime = time.time()

for idx, (url, date) in enumerate(zip(urls, dates)):
    _thread.start_new_thread(get_news, (url, date, idx, startTime))

w/ multi-thread:

[No.3]  accumulated time: 1.0591027736663818
[No.1]  accumulated time: 1.1277062892913818
[No.4]  accumulated time: 1.2443935871124268
[No.2]  accumulated time: 1.2783033847808838
[No.0]  accumulated time: 1.4958300590515137
